In [1]:
!pip install azureml-dataprep

In [2]:
import azureml.dataprep as dprep

## Load Data

Download two different NYC taxi data sets into dataflow objects. The datasets have slightly different fields. The auto_read_file() method automatically recognizes the input file type.

In [3]:
from IPython.display import display
dataset_root = "https://dprepdata.blob.core.windows.net/demo"

green_path = "/".join([dataset_root, "green-small/*"])
yellow_path = "/".join([dataset_root, "yellow-small/*"])

green_df_raw = dprep.read_csv(path=green_path, header=dprep.PromoteHeadersMode.GROUPED)
# auto_read_file automatically identifies and parses the file type, which is useful when you don't know the file type.
yellow_df_raw = dprep.auto_read_file(path=yellow_path)

display(green_df_raw.head(5))
display(yellow_df_raw.head(5))

,VendorID,lpep_pickup_datetime,Lpep_dropoff_datetime,Store_and_fwd_flag,RateCodeID,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Passenger_count,...,Extra,MTA_tax,Tip_amount,Tolls_amount,Ehail_fee,Total_amount,Payment_type,Trip_type,Column21,Column22
0,,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,2,2013-08-01 08:14:37,2013-08-01 09:09:06,N,1,0,0,0,0,1,...,0,0,0,0,,21.25,2,,,
2,2,2013-08-01 09:13:00,2013-08-01 11:38:00,N,1,0,0,0,0,2,...,0,0.5,0,0,,75,2,,,
3,2,2013-08-01 09:48:00,2013-08-01 09:49:00,N,5,0,0,0,0,1,...,0.1,0,0,1,,2.1,2,,,
4,2,2013-08-01 10:38:35,2013-08-01 10:38:51,N,1,0,0,0,0,1,...,0,0,0,0,,3.25,2,,,


,vendor_name,Trip_Pickup_DateTime,Trip_Dropoff_DateTime,Passenger_Count,Trip_Distance,Start_Lon,Start_Lat,Rate_Code,store_and_forward,End_Lon,End_Lat,Payment_Type,Fare_Amt,surcharge,mta_tax,Tip_Amt,Tolls_Amt,Total_Amt
0,,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,VTS,2009-01-04 02:52:00,2009-01-04 03:02:00,1,2.6299999999999999,-73.991956999999999,40.721567,,,-73.993803,40.695922000000003,CASH,8.9000000000000004,0.5,,0,0,9.4000000000000004
2,VTS,2009-01-04 03:31:00,2009-01-04 03:38:00,3,4.5499999999999998,-73.982101999999998,40.736289999999997,,,-73.955849999999998,40.768030000000003,Credit,12.1,0.5,,2,0,14.6
3,VTS,2009-01-03 15:43:00,2009-01-03 15:57:00,5,10.35,-74.002587000000005,40.739747999999999,,,-73.869983000000005,40.770225000000003,Credit,23.699999999999999,0,,4.7400000000000002,0,28.440000000000001
4,DDS,2009-01-01 20:52:58,2009-01-01 21:14:00,1,5,-73.974266999999998,40.790954999999997,,,-73.996557999999993,40.731848999999997,CREDIT,14.9,0.5,,3.0499999999999998,0,18.449999999999999


A Dataflow object is similar to a dataframe

## Cleanse data

The drop_if_all_null variable is used to delete records where all fields are null. The useful_columns variable holds an array of column descriptions that are kept in each dataflow.

In [4]:
all_columns = dprep.ColumnSelector(term=".*", use_regex=True)
drop_if_all_null = [all_columns, dprep.ColumnRelationship(dprep.ColumnRelationship.ALL)]
useful_columns = [
    "cost", "distance", "dropoff_datetime", "dropoff_latitude", "dropoff_longitude",
    "passengers", "pickup_datetime", "pickup_latitude", "pickup_longitude", "store_forward", "vendor"
]

 replace_na(), drop_nulls(), and keep_columns() functions are used to transform the green taxi data set so that it can be merged to yellow taxi dataset

Also rename all the columns in the dataframe to match the names in the useful_columns variable.

In [6]:
green_df = (green_df_raw
    .replace_na(columns=all_columns)
    .drop_nulls(*drop_if_all_null)
    .rename_columns(column_pairs={
        "VendorID": "vendor",
        "lpep_pickup_datetime": "pickup_datetime",
        "Lpep_dropoff_datetime": "dropoff_datetime",
        "lpep_dropoff_datetime": "dropoff_datetime",
        "Store_and_fwd_flag": "store_forward",
        "store_and_fwd_flag": "store_forward",
        "Pickup_longitude": "pickup_longitude",
        "Pickup_latitude": "pickup_latitude",
        "Dropoff_longitude": "dropoff_longitude",
        "Dropoff_latitude": "dropoff_latitude",
        "Passenger_count": "passengers",
        "Fare_amount": "cost",
        "Trip_distance": "distance"
     })
    .keep_columns(columns=useful_columns))
green_df.head(5)

,vendor,pickup_datetime,dropoff_datetime,store_forward,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passengers,distance,cost
0,2,2013-08-01 08:14:37,2013-08-01 09:09:06,N,0,0,0,0,1,.00,21.25
1,2,2013-08-01 09:13:00,2013-08-01 11:38:00,N,0,0,0,0,2,.00,74.5
2,2,2013-08-01 09:48:00,2013-08-01 09:49:00,N,0,0,0,0,1,.00,1
3,2,2013-08-01 10:38:35,2013-08-01 10:38:51,N,0,0,0,0,1,.00,3.25
4,2,2013-08-01 11:51:45,2013-08-01 12:03:52,N,0,0,0,0,1,.00,8.5


 the same transformation steps are run on the yellow taxi data

In [7]:
yellow_df = (yellow_df_raw
    .replace_na(columns=all_columns)
    .drop_nulls(*drop_if_all_null)
    .rename_columns(column_pairs={
        "vendor_name": "vendor",
        "VendorID": "vendor",
        "vendor_id": "vendor",
        "Trip_Pickup_DateTime": "pickup_datetime",
        "tpep_pickup_datetime": "pickup_datetime",
        "Trip_Dropoff_DateTime": "dropoff_datetime",
        "tpep_dropoff_datetime": "dropoff_datetime",
        "store_and_forward": "store_forward",
        "store_and_fwd_flag": "store_forward",
        "Start_Lon": "pickup_longitude",
        "Start_Lat": "pickup_latitude",
        "End_Lon": "dropoff_longitude",
        "End_Lat": "dropoff_latitude",
        "Passenger_Count": "passengers",
        "passenger_count": "passengers",
        "Fare_Amt": "cost",
        "fare_amount": "cost",
        "Trip_Distance": "distance",
        "trip_distance": "distance"
    })
    .keep_columns(columns=useful_columns))
yellow_df.head(5)

,vendor,pickup_datetime,dropoff_datetime,passengers,distance,pickup_longitude,pickup_latitude,store_forward,dropoff_longitude,dropoff_latitude,cost
0,VTS,2009-01-04 02:52:00,2009-01-04 03:02:00,1,2.6299999999999999,-73.991956999999999,40.721567,None,-73.993803,40.695922000000003,8.9000000000000004
1,VTS,2009-01-04 03:31:00,2009-01-04 03:38:00,3,4.5499999999999998,-73.982101999999998,40.736289999999997,None,-73.955849999999998,40.768030000000003,12.1
2,VTS,2009-01-03 15:43:00,2009-01-03 15:57:00,5,10.35,-74.002587000000005,40.739747999999999,None,-73.869983000000005,40.770225000000003,23.699999999999999
3,DDS,2009-01-01 20:52:58,2009-01-01 21:14:00,1,5,-73.974266999999998,40.790954999999997,None,-73.996557999999993,40.731848999999997,14.9
4,DDS,2009-01-24 16:18:23,2009-01-24 16:24:56,1,0.40000000000000002,-74.001580000000004,40.719382000000003,None,-74.008377999999993,40.720350000000003,3.7000000000000002


 append_rows() functionis applied on the green taxi data to append the yellow taxi data. A new combined dataframe is created.

In [8]:
combined_df = green_df.append_rows([yellow_df])

The latitude and longitude columns are converted to decimal format using the TypeConverter function. In order to get the the description of only these columns the get_profile() and keep_columns() function are used 

In [9]:
decimal_type = dprep.TypeConverter(data_type=dprep.FieldType.DECIMAL)
combined_df = combined_df.set_column_types(type_conversions={
    "pickup_longitude": decimal_type,
    "pickup_latitude": decimal_type,
    "dropoff_longitude": decimal_type,
    "dropoff_latitude": decimal_type
})
combined_df.keep_columns(columns=[
    "pickup_longitude", "pickup_latitude",
    "dropoff_longitude", "dropoff_latitude"
]).get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent missing,Error Count,Empty count,0.1% Quantile,1% Quantile,5% Quantile,25% Quantile,50% Quantile,75% Quantile,95% Quantile,99% Quantile,99.9% Quantile,Mean,Standard Deviation,Variance,Skewness,Kurtosis
pickup_longitude,FieldType.DECIMAL,-115.179337,0.000000,7722.0,0.0,7722.0,0.0,0.0,0.0,-80.110658,-73.961885,-73.962354,-73.947823,-73.921860,-73.846196,0.000000,0.000000,0.000000,-68.833579,18.792672,353.164515,3.369970,9.475822
pickup_latitude,FieldType.DECIMAL,0.000000,40.919121,7722.0,0.0,7722.0,0.0,0.0,0.0,0.000000,40.684513,40.683247,40.721145,40.756218,40.804345,40.849468,40.870242,40.892072,37.936742,10.345967,107.039038,-3.391034,9.507615
dropoff_longitude,FieldType.DECIMAL,-115.179337,0.000000,7722.0,0.0,7722.0,0.0,0.0,0.0,-80.091993,-73.984740,-73.985831,-73.956148,-73.928934,-73.866010,0.000000,0.000000,0.000000,-68.896978,18.696526,349.560067,3.393537,9.637675
dropoff_latitude,FieldType.DECIMAL,0.000000,41.008934,7722.0,0.0,7722.0,0.0,0.0,0.0,0.000000,40.663088,40.661844,40.718004,40.756568,40.784694,40.852264,40.879800,40.925400,37.963774,10.290780,105.900157,-3.414918,9.670386


In [10]:
latlong_filtered_df = (combined_df
    .drop_nulls(
        columns=["pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude"],
        column_relationship=dprep.ColumnRelationship(dprep.ColumnRelationship.ANY)
    )
    .filter(dprep.f_and(
        dprep.col("pickup_longitude") <= -73.72,
        dprep.col("pickup_longitude") >= -74.09,
        dprep.col("pickup_latitude") <= 40.88,
        dprep.col("pickup_latitude") >= 40.53,
        dprep.col("dropoff_longitude") <= -73.72,
        dprep.col("dropoff_longitude") >= -74.09,
        dprep.col("dropoff_latitude") <= 40.88,
        dprep.col("dropoff_latitude") >= 40.53
    )))
latlong_filtered_df.keep_columns(columns=[
    "pickup_longitude", "pickup_latitude",
    "dropoff_longitude", "dropoff_latitude"
]).get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent missing,Error Count,Empty count,0.1% Quantile,1% Quantile,5% Quantile,25% Quantile,50% Quantile,75% Quantile,95% Quantile,99% Quantile,99.9% Quantile,Mean,Standard Deviation,Variance,Skewness,Kurtosis
pickup_longitude,FieldType.DECIMAL,-74.078156,-73.736481,7059.0,0.0,7059.0,0.0,0.0,0.0,-74.078006,-73.962538,-73.962717,-73.948994,-73.927816,-73.865688,-73.830361,-73.824080,-73.769338,-73.913865,0.048711,0.002373,0.402697,-0.613516
pickup_latitude,FieldType.DECIMAL,40.575485,40.879852,7059.0,0.0,7059.0,0.0,0.0,0.0,40.633259,40.713189,40.713110,40.721369,40.758147,40.805117,40.848848,40.867566,40.877787,40.765226,0.048348,0.002338,0.228088,-0.598862
dropoff_longitude,FieldType.DECIMAL,-74.085747,-73.720871,7059.0,0.0,7059.0,0.0,0.0,0.0,-74.078187,-73.985679,-73.986376,-73.959024,-73.936762,-73.884944,-73.815567,-73.776707,-73.733476,-73.920718,0.055961,0.003132,0.648649,0.022914
dropoff_latitude,FieldType.DECIMAL,40.583530,40.879734,7059.0,0.0,7059.0,0.0,0.0,0.0,40.600361,40.695508,40.693780,40.727604,40.758163,40.788390,40.850519,40.867857,40.878703,40.759487,0.050462,0.002546,0.048418,-0.036880


## Split and rename columns

The store_forward columnis a boolean flag that is Y when the taxi did not have a connection to the server after the trip, and thus had to store the trip data in memory, and later forward it to the server when connected.

In [11]:
latlong_filtered_df.keep_columns(columns='store_forward').get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent missing,Error Count,Empty count,0.1% Quantile,1% Quantile,5% Quantile,25% Quantile,50% Quantile,75% Quantile,95% Quantile,99% Quantile,99.9% Quantile,Mean,Standard Deviation,Variance,Skewness,Kurtosis
store_forward,FieldType.STRING,N,Y,7059.0,99.0,6960.0,0.014025,0.0,0.0,,,,,,,,,,,,,,


Notice that the data profile output in the store_forward column shows that the data is inconsistent and there are missing or null values. Use the replace() and fill_nulls() functions to replace these values with the string "N"

In [12]:
replaced_stfor_vals_df = latlong_filtered_df.replace(columns="store_forward", find="0", replace_with="N").fill_nulls("store_forward", "N")

Execute the replace function on the distance field. The function reformats distance values that are incorrectly labeled as .00, and fills any nulls with zeros. Convert the distance field to numerical format. These incorrect data points are likely anomalies in the data collection system on the taxi cabs.


In [13]:
replaced_distance_vals_df = replaced_stfor_vals_df.replace(columns="distance", find=".00", replace_with=0).fill_nulls("distance", 0)
replaced_distance_vals_df = replaced_distance_vals_df.to_number(["distance"])

Split the pickup and dropoff datetime values into the respective date and time columns. Use the split_column_by_example() function to make the split. In this case, the optional example parameter of the split_column_by_example() function is omitted. Therefore, the function automatically determines where to split based on the data.

In [14]:
time_split_df = (replaced_distance_vals_df
    .split_column_by_example(source_column="pickup_datetime")
    .split_column_by_example(source_column="dropoff_datetime"))
time_split_df.head(5)

,vendor,pickup_datetime,pickup_datetime_1,pickup_datetime_2,dropoff_datetime,dropoff_datetime_1,dropoff_datetime_2,store_forward,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passengers,distance,cost
0,2,2013-08-01 17:22:00,2013-08-01,17:22:00,2013-08-01 17:22:00,2013-08-01,17:22:00,N,-73.937767,40.758480,-73.937767,40.758480,1,0.0,2.5
1,2,2013-08-01 17:24:00,2013-08-01,17:24:00,2013-08-01 17:25:00,2013-08-01,17:25:00,N,-73.937927,40.757843,-73.937927,40.757843,1,0.0,2.5
2,2,2013-08-06 06:51:19,2013-08-06,06:51:19,2013-08-06 06:51:36,2013-08-06,06:51:36,N,-73.937721,40.758404,-73.937721,40.758369,1,0.0,3.3
3,2,2013-08-06 13:26:34,2013-08-06,13:26:34,2013-08-06 13:26:57,2013-08-06,13:26:57,N,-73.937691,40.758419,-73.937790,40.758358,1,0.0,3.3
4,2,2013-08-06 13:27:53,2013-08-06,13:27:53,2013-08-06 13:28:08,2013-08-06,13:28:08,N,-73.937805,40.758396,-73.937775,40.758450,1,0.0,3.3


Rename the columns generated by the split_column_by_example() function to use meaningful names.

In [15]:
renamed_col_df = (time_split_df
    .rename_columns(column_pairs={
        "pickup_datetime_1": "pickup_date",
        "pickup_datetime_2": "pickup_time",
        "dropoff_datetime_1": "dropoff_date",
        "dropoff_datetime_2": "dropoff_time"
    }))
renamed_col_df.head(5)

,vendor,pickup_datetime,pickup_date,pickup_time,dropoff_datetime,dropoff_date,dropoff_time,store_forward,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passengers,distance,cost
0,2,2013-08-01 17:22:00,2013-08-01,17:22:00,2013-08-01 17:22:00,2013-08-01,17:22:00,N,-73.937767,40.758480,-73.937767,40.758480,1,0.0,2.5
1,2,2013-08-01 17:24:00,2013-08-01,17:24:00,2013-08-01 17:25:00,2013-08-01,17:25:00,N,-73.937927,40.757843,-73.937927,40.757843,1,0.0,2.5
2,2,2013-08-06 06:51:19,2013-08-06,06:51:19,2013-08-06 06:51:36,2013-08-06,06:51:36,N,-73.937721,40.758404,-73.937721,40.758369,1,0.0,3.3
3,2,2013-08-06 13:26:34,2013-08-06,13:26:34,2013-08-06 13:26:57,2013-08-06,13:26:57,N,-73.937691,40.758419,-73.937790,40.758358,1,0.0,3.3
4,2,2013-08-06 13:27:53,2013-08-06,13:27:53,2013-08-06 13:28:08,2013-08-06,13:28:08,N,-73.937805,40.758396,-73.937775,40.758450,1,0.0,3.3


Call the get_profile() function to see the full summary statistics after all cleansing steps.

In [16]:
renamed_col_df.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent missing,Error Count,Empty count,0.1% Quantile,1% Quantile,5% Quantile,25% Quantile,50% Quantile,75% Quantile,95% Quantile,99% Quantile,99.9% Quantile,Mean,Standard Deviation,Variance,Skewness,Kurtosis
vendor,FieldType.STRING,1,VTS,7059.0,0.0,7059.0,0.0,0.0,0.0,,,,,,,,,,,,,,
pickup_datetime,FieldType.STRING,2009-01-01 20:52:58,2013-08-31 23:59:14,7059.0,0.0,7059.0,0.0,0.0,0.0,,,,,,,,,,,,,,
pickup_date,FieldType.STRING,2009-01-01,2013-08-31,7059.0,0.0,7059.0,0.0,0.0,0.0,,,,,,,,,,,,,,
pickup_time,FieldType.STRING,00:00:48,23:59:48,7059.0,0.0,7059.0,0.0,0.0,0.0,,,,,,,,,,,,,,
dropoff_datetime,FieldType.STRING,2009-01-01 21:14:00,2013-09-01 00:25:07,7059.0,0.0,7059.0,0.0,0.0,0.0,,,,,,,,,,,,,,
dropoff_date,FieldType.STRING,2009-01-01,2013-09-01,7059.0,0.0,7059.0,0.0,0.0,0.0,,,,,,,,,,,,,,
dropoff_time,FieldType.STRING,00:00:00,23:59:45,7059.0,0.0,7059.0,0.0,0.0,0.0,,,,,,,,,,,,,,
store_forward,FieldType.STRING,N,Y,7059.0,0.0,7059.0,0.0,0.0,0.0,,,,,,,,,,,,,,
pickup_longitude,FieldType.DECIMAL,-74.0782,-73.7365,7059.0,0.0,7059.0,0.0,0.0,0.0,-74.078,-73.9625,-73.9627,-73.949,-73.9278,-73.8657,-73.8304,-73.8241,-73.7693,-73.9139,0.0487111,0.00237277,0.402697,-0.613516
pickup_latitude,FieldType.DECIMAL,40.5755,40.8799,7059.0,0.0,7059.0,0.0,0.0,0.0,40.6333,40.7132,40.7131,40.7214,40.7581,40.8051,40.8488,40.8676,40.8778,40.7652,0.0483485,0.00233758,0.228088,-0.598862


## Transform data

generating a new feature for the weekday will help establish a relationship between the day of the week and the taxi fare price, which is often more expensive on certain days of the week due to high demand.
To get the day of the week value, use the derive_column_by_example() function. The function takes an array parameter of example objects that define the input data, and the preferred output. The function automatically determines your preferred transformation. For the pickup and dropoff time columns, split the time into the hour, minute, and second by using the split_column_by_example() function with no example parameter.

In [17]:
transformed_features_df = (renamed_col_df
    .derive_column_by_example(
        source_columns="pickup_date",
        new_column_name="pickup_weekday",
        example_data=[("2009-01-04", "Sunday"), ("2013-08-22", "Thursday")]
    )
    .derive_column_by_example(
        source_columns="dropoff_date",
        new_column_name="dropoff_weekday",
        example_data=[("2013-08-22", "Thursday"), ("2013-11-03", "Sunday")]
    )

    .split_column_by_example(source_column="pickup_time")
    .split_column_by_example(source_column="dropoff_time")
    # The following two calls to split_column_by_example reference the column names generated from the previous two calls.
    .split_column_by_example(source_column="pickup_time_1")
    .split_column_by_example(source_column="dropoff_time_1")
    .drop_columns(columns=[
        "pickup_date", "pickup_time", "dropoff_date", "dropoff_time",
        "pickup_date_1", "dropoff_date_1", "pickup_time_1", "dropoff_time_1"
    ])

    .rename_columns(column_pairs={
        "pickup_date_2": "pickup_month",
        "pickup_date_3": "pickup_monthday",
        "pickup_time_1_1": "pickup_hour",
        "pickup_time_1_2": "pickup_minute",
        "pickup_time_2": "pickup_second",
        "dropoff_date_2": "dropoff_month",
        "dropoff_date_3": "dropoff_monthday",
        "dropoff_time_1_1": "dropoff_hour",
        "dropoff_time_1_2": "dropoff_minute",
        "dropoff_time_2": "dropoff_second"
    }))

transformed_features_df.head(5)

,vendor,pickup_datetime,pickup_weekday,pickup_hour,pickup_minute,pickup_second,dropoff_datetime,dropoff_weekday,dropoff_hour,dropoff_minute,dropoff_second,store_forward,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passengers,distance,cost
0,2,2013-08-01 17:22:00,Thursday,17,22,00,2013-08-01 17:22:00,Thursday,17,22,00,N,-73.937767,40.758480,-73.937767,40.758480,1,0.0,2.5
1,2,2013-08-01 17:24:00,Thursday,17,24,00,2013-08-01 17:25:00,Thursday,17,25,00,N,-73.937927,40.757843,-73.937927,40.757843,1,0.0,2.5
2,2,2013-08-06 06:51:19,Tuesday,06,51,19,2013-08-06 06:51:36,Tuesday,06,51,36,N,-73.937721,40.758404,-73.937721,40.758369,1,0.0,3.3
3,2,2013-08-06 13:26:34,Tuesday,13,26,34,2013-08-06 13:26:57,Tuesday,13,26,57,N,-73.937691,40.758419,-73.937790,40.758358,1,0.0,3.3
4,2,2013-08-06 13:27:53,Tuesday,13,27,53,2013-08-06 13:28:08,Tuesday,13,28,08,N,-73.937805,40.758396,-73.937775,40.758450,1,0.0,3.3


In [18]:
processed_df = transformed_features_df.drop_columns(columns=["pickup_datetime", "dropoff_datetime"])

In [19]:
#Use the type inference functionality to automatically check the data type of each field, and display the inference results.
type_infer = processed_df.builders.set_column_types()
type_infer.learn()
type_infer

Column types conversion candidates:
'dropoff_minute': [FieldType.INTEGER],
'distance': [FieldType.DECIMAL],
'pickup_longitude': [FieldType.DECIMAL],
'pickup_latitude': [FieldType.DECIMAL],
'dropoff_hour': [FieldType.INTEGER],
'pickup_hour': [FieldType.INTEGER],
'passengers': [FieldType.INTEGER],
'pickup_second': [FieldType.INTEGER],
'dropoff_latitude': [FieldType.DECIMAL],
'dropoff_second': [FieldType.INTEGER],
'dropoff_longitude': [FieldType.DECIMAL],
'pickup_weekday': [FieldType.STRING],
'store_forward': [FieldType.BOOLEAN],
'dropoff_weekday': [FieldType.STRING],
'pickup_minute': [FieldType.INTEGER],
'cost': [FieldType.DECIMAL],
'vendor': [FieldType.STRING]

In [21]:
#The inference results look correct based on the data. Now apply the type conversions to the dataflow.
type_converted_df = type_infer.to_dataflow()
type_converted_df.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent missing,Error Count,Empty count,0.1% Quantile,1% Quantile,5% Quantile,25% Quantile,50% Quantile,75% Quantile,95% Quantile,99% Quantile,99.9% Quantile,Mean,Standard Deviation,Variance,Skewness,Kurtosis
vendor,FieldType.STRING,1,VTS,7059.0,0.0,7059.0,0.0,0.0,0.0,,,,,,,,,,,,,,
pickup_weekday,FieldType.STRING,Friday,Wednesday,7059.0,0.0,7059.0,0.0,0.0,0.0,,,,,,,,,,,,,,
pickup_hour,FieldType.INTEGER,0,23,7059.0,0.0,7059.0,0.0,0.0,0.0,0,3.7504,3,10,16,19,22.1376,23,23,14.2326,6.34926,40.3131,-0.693335,-0.459336
pickup_minute,FieldType.INTEGER,0,59,7059.0,0.0,7059.0,0.0,0.0,0.0,0,5,5,14,29.4903,44.8062,56.7184,59,59,29.4635,17.4396,304.14,0.00440324,-1.20458
pickup_second,FieldType.INTEGER,0,59,7059.0,0.0,7059.0,0.0,0.0,0.0,0,5.02995,5,14.7086,29.9385,44.5977,56.9191,59,59,29.6225,17.3868,302.302,-0.0227466,-1.19409
dropoff_weekday,FieldType.STRING,Friday,Wednesday,7059.0,0.0,7059.0,0.0,0.0,0.0,,,,,,,,,,,,,,
dropoff_hour,FieldType.INTEGER,0,23,7059.0,0.0,7059.0,0.0,0.0,0.0,0,3,3,10,16,19,22.0764,23,23,14.1815,6.45578,41.677,-0.691001,-0.500215
dropoff_minute,FieldType.INTEGER,0,59,7059.0,0.0,7059.0,0.0,0.0,0.0,0,5.14087,5,14,28.9957,44.0043,56.8986,59,59,29.353,17.4241,303.598,0.0142562,-1.21531
dropoff_second,FieldType.INTEGER,0,59,7059.0,0.0,7059.0,0.0,0.0,0.0,0,5,5,14.8268,29.6546,45.2547,56,59,59,29.7923,17.481,305.585,-0.0281313,-1.21965
store_forward,FieldType.BOOLEAN,False,True,7059.0,0.0,7059.0,0.0,0.0,0.0,,,,,,,,,,,,,,


Before you package the dataflow, run two final filters on the data set. To eliminate incorrectly captured data points, filter the dataflow on records where both the cost and distance variable values are greater than zero. This step will significantly improve machine learning model accuracy, because data points with a zero cost or distance represent major outliers that throw off prediction accuracy.

In [22]:
final_df = type_converted_df.filter(dprep.col("distance") > 0)
final_df = final_df.filter(dprep.col("cost") > 0)

You now have a fully transformed and prepared dataflow object to use in a machine learning model. The SDK includes object serialization functionality, which is used as shown in the following code.

In [23]:
import os
file_path = os.path.join(os.getcwd(), "dflows.dprep")
final_df.save(file_path)